###  3.4 神经网络

变量：tf.Variable，变量的作用是保存和更新神经网络中的参数，需要初始化。

变量的声明函数是一个运算，该运算的结果是一个张量。所以变量只是一种特殊的张量。

示例：通过变量实现神经网络的参数，并实现前向传播

In [2]:
import tensorflow as tf

# 声明两个变量。通过seed参数设定随机种子
w1= tf.Variable(tf.random_normal([2, 3], stddev=1, seed=1))
w2= tf.Variable(tf.random_normal([3, 1], stddev=1, seed=1))

# 将入参定义为参量
x = tf.constant([[0.7, 0.9]])

# 通过前向传播算法获得神经网络的输出
a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

sess = tf.Session()
# 因为w1和w2还没初始化，所以不能直接通过 sess.run(y)来获取y的值
sess.run(w1.initializer)  # 初始化w1
sess.run(w2.initializer)  # 初始化w2
print(sess.run(y))  
sess.close()

[[3.957578]]


更便捷的方式完成变量初始化：tf.initialize_all_variable 函数

优点：不需要一个有一个初始化变量；自动处理变量之间的依赖关系。

```
init_op = tf.initialize_all_variable()
sess.run(init_op)
```

**placeholder** 

每生成一个常量，tf都会在计算图中增加一个节点。训练中每轮迭代中选取的数据都要通过常量来表示。

placeholder：相当于定义了一个位置，这个位置中的数据在程序运行时再指定。这样程序中就不需要生成大量常量来提供输入数据，而只需要将数据通过placeholder传入tf计算图。

示例：通过placeholder 实现前向传播

In [7]:
import tensorflow as tf

w1= tf.Variable(tf.random_normal([2, 3], stddev=1))
w2= tf.Variable(tf.random_normal([3, 1], stddev=1))

# 定义placeholder作为存放输入数据的地方
# 维度shape不一定要定义。但如果维度是确定的，给出维度可以降低出错的概率
#x = tf.placeholder(tf.float32, shape=(1,2), name="input")
x = tf.placeholder(tf.float32, shape=(3,2), name="input")
a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

sess = tf.Session()
init_op = tf.initialize_all_variables()
sess.run(init_op)

# InvalidArgumentError: You must feed a value for placeholder 
# tensor 'input' with dtype float and shape [1,2]
# print(sess.run(y))

# feed_dict 来指定x的取值
#print(sess.run(y, feed_dict={x: [[0.7, 0.9]]}))
print(sess.run(y, feed_dict={x: [[0.7,0.9], [0.1,0.4], [0.5,0.8]]}))
sess.close()

[[1.2116898 ]
 [0.36900944]
 [0.978915  ]]


In [6]:
x = tf.placeholder(tf.float32, shape=(1, 2), name="input")
a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

# 使用 with 的方式
with tf.Session().as_default() as sess:
    init_op = tf.global_variables_initializer()
    print(sess.run(init_op))
    # 两种输出结果的方法
    print(sess.run(y, feed_dict={x: [[0.7, 0.9]]}))
    print(y.eval(feed_dict={x: [[.7,.9]]}))

None
[[1.4607782]]
[[1.4607782]]


得到一个batch的前向传播结果后，需要一个损失函数来刻画当前的预测值和真实答案之间的差距。然后通过反向传播算法来调整神经网络参数的取值使得差距可以被缩小。